# Numbers

[Numeric types](https://go.dev/ref/spec#Numeric_types) section in [The Go Programming Language Specification](https://go.dev/ref/)

Before we start, check Go version:

In [47]:
%%
fmt.Println(runtime. Version())

go1.21.1


## Integers

Go has 8 fixed-size integer types and two widely used aliases. From the [specs](https://go.dev/ref/spec#Numeric_types):

```go
uint8       // unsigned  8-bit integers (0 to 255)
uint16      // unsigned 16-bit integers (0 to 65535)
uint32      // unsigned 32-bit integers (0 to 4294967295)
uint64      // unsigned 64-bit integers (0 to 18446744073709551615)

int8        // signed  8-bit integers (-128 to 127)
int16       // signed 16-bit integers (-32768 to 32767)
int32       // signed 32-bit integers (-2147483648 to 2147483647)
int64       // signed 64-bit integers (-9223372036854775808 to 9223372036854775807)

float32     // IEEE 754 32-bit floating-point numbers
float64     // IEEE 754 64-bit floating-point numbers

complex64   // complex numbers with float32 real and imaginary parts
complex128  // complex numbers with float64 real and imaginary parts

byte        alias for uint8
rune        alias for int32
```

Go also has 3 integer types with implementation-dependent sizes:

```go
uint     // either 32 or 64 bits
int      // same size as uint
uintptr  // an unsigned integer large enough to store the uninterpreted bits of a pointer value
```

The integer types all wrap-around: if you add 1 to the maximum value, you get the minimum value, and vice-versa.

Examples:

In [11]:
%%
var u uint8
fmt.Println(u)
fmt.Println(u - 1)

0
255


In [15]:
%%
var u uint8 = 255
fmt.Println(u)
fmt.Println(u + 1)

255
0


In [16]:
%%
var i int8 = 127
fmt.Println(i + 1)

-128


In [17]:
%%
var i int8 = -128
fmt.Println(i - 1)

127


In [23]:
func pow2(exponent uint64) (result uint64) {
	result = 1
    var i uint64
	for i = 0; i < exponent; i++ {
		result *= 2
	}
	return result
}

In [32]:
%%
fmt.Println(pow2(63))
fmt.Println(pow2(64))
fmt.Println(pow2(64)-1)
const uint64max = 18446744073709551615  // https://go.dev/ref/spec#Numeric_types
fmt.Println(pow2(64)-1 == uint64max) 

9223372036854775808
0
18446744073709551615
true


### What's the size of my int?

In [34]:
%%
fmt.Println(bits.UintSize)

64


In [35]:
%%
uintSize := 32 << (^uint(0) >> 32 & 1) // 32 or 64 
fmt.Println(uintSize)

64


In [39]:
%%
fmt.Println(^uint32(0))

4294967295


In [53]:
%%
for i := 0; i < 9; i++ {
    j := ^uint8(0) >> i
    fmt.Printf("%d %3d %08b\n", i, j, j)
}


0 255 11111111
1 127 01111111
2  63 00111111
3  31 00011111
4  15 00001111
5   7 00000111
6   3 00000011
7   1 00000001
8   0 00000000
